In [ ]:
# !pip uninstall -y langchain langchain-core langchain-community langchain-huggingface
# !pip install -q langchain==0.1.14 langchain-community==0.0.34 langchain-core==0.1.50 langchain-huggingface==0.2.0 sentence-transformers transformers llama-cpp-python tf-keras faiss-cpu numpy==1.26.4 pandas==1.5.3 scikit-learn==1.2.2 ml-dtypes==0.4.0 flask huggingface-hub einops protobuf beautifulsoup4 torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu126
!pip install -q -U langchain langchain-core langchain-community langchain-huggingface sentence-transformers transformers llama-cpp-python tf-keras 
!pip install numpy==1.26.4 pandas==1.5.3 scikit-learn==1.2.2 ml-dtypes==0.4.0 flask huggingface-hub einops protobuf beautifulsoup4 qdrant-client torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121 # Hoặc cu118, cu126 tùy theo môi trường Kaggle và driver

In [ ]:
import torch
from langchain_huggingface import HuggingFaceEmbeddings
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

# Qdrant specific
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant # Import Qdrant vector store

# Định nghĩa đường dẫn và tên collection cho Qdrant trong môi trường Kaggle
# Thư mục /kaggle/working/ là nơi có thể ghi dữ liệu.
qdrant_kaggle_path = "/kaggle/working/qdrant_kaggle_store"
qdrant_collection_name_kaggle = "huit_admissions_collection_kaggle" # Có thể giữ tên giống local hoặc thêm suffix

embedding_model_name = "sentence-transformers/all-MiniLM-L12-v2"
device_type_kaggle = "cuda" if torch.cuda.is_available() else "cpu"
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs= {"device": device_type_kaggle}
)

print(f"CUDA Available on Kaggle: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")
    # Kiểm tra VRAM (ví dụ, không phải là cách chính xác nhất nhưng cho ý tưởng)
    # t = torch.cuda.get_device_properties(0)
    # print(f"Total VRAM: {t.total_memory / (1024**3):.2f} GB")
print(f"Embedding model device: {embedding_model.client.device}")

In [ ]:
# Cell 3: Chuẩn bị dữ liệu (Copy file từ Kaggle Datasets)
import shutil
import os

# Điều chỉnh các đường dẫn này cho phù hợp với dataset của bạn trên Kaggle
KAGGLE_INPUT_DATA_DIR = "/kaggle/input/dataset-tuvantuyensinh" # Ví dụ: /kaggle/input/your-dataset-slug
KAGGLE_INPUT_MODEL_DIR = "/kaggle/input/vinallama/gguf/default/1" # Ví dụ: /kaggle/input/your-model-slug/path/to/gguf

# Thư mục đích trong không gian làm việc của Kaggle (có thể ghi)
local_data_dir_kaggle = "/kaggle/working/data"
local_models_dir_kaggle = "/kaggle/working/models"

os.makedirs(local_data_dir_kaggle, exist_ok=True)
os.makedirs(local_models_dir_kaggle, exist_ok=True)

# Copy file .txt từ dataset đầu vào
if os.path.exists(KAGGLE_INPUT_DATA_DIR):
    print(f"📁 Files trong dataset đầu vào ({KAGGLE_INPUT_DATA_DIR}): {os.listdir(KAGGLE_INPUT_DATA_DIR)}")
    for file_name in os.listdir(KAGGLE_INPUT_DATA_DIR):
        if file_name.endswith(".txt"):
            source_file = os.path.join(KAGGLE_INPUT_DATA_DIR, file_name)
            dest_file = os.path.join(local_data_dir_kaggle, file_name)
            shutil.copy(source_file, dest_file)
    print(f"✅ Đã copy các file .txt vào {local_data_dir_kaggle}")
else:
    print(f"[!] Đường dẫn dataset đầu vào {KAGGLE_INPUT_DATA_DIR} không tồn tại. Hãy đảm bảo bạn đã thêm dataset vào notebook.")

# Copy model .gguf
model_gguf_filename = "vinallama-7b-chat_q5_0.gguf" # Tên file model của bạn
gguf_source_path_kaggle = os.path.join(KAGGLE_INPUT_MODEL_DIR, model_gguf_filename)
gguf_dest_path_kaggle = os.path.join(local_models_dir_kaggle, model_gguf_filename)
model_GGUF_path_kaggle = None # Sẽ được cập nhật

if os.path.exists(gguf_source_path_kaggle):
    print(f"📁 File model GGUF tìm thấy tại: {gguf_source_path_kaggle}")
    shutil.copy(gguf_source_path_kaggle, gguf_dest_path_kaggle)
    model_GGUF_path_kaggle = gguf_dest_path_kaggle
    print(f"✅ Đã copy model {model_gguf_filename} vào {local_models_dir_kaggle}")
else:
    print(f"[!] Model GGUF không tìm thấy tại {gguf_source_path_kaggle}.")
    print("Đang thử tải model từ HuggingFace Hub (cần bật Internet cho notebook)...")
    from huggingface_hub import hf_hub_download
    try:
        # Đảm bảo repo_id và filename là chính xác
        downloaded_path = hf_hub_download(
            repo_id="vilm/vinallama-7b-chat-GGUF",
            filename=model_gguf_filename,
            local_dir=local_models_dir_kaggle,
            local_dir_use_symlinks=False # Tốt nhất cho Kaggle
        )
        model_GGUF_path_kaggle = downloaded_path # hf_hub_download trả về đường dẫn đầy đủ
        print(f"✅ Đã tải model {model_gguf_filename} vào {model_GGUF_path_kaggle} từ Hub.")
    except Exception as e:
        print(f"[!] Lỗi tải model từ Hub: {e}. Model sẽ không khả dụng.")

if not model_GGUF_path_kaggle or not os.path.exists(model_GGUF_path_kaggle):
     print(f"[!] QUAN TRỌNG: Không tìm thấy hoặc không thể tải file model GGUF {model_gguf_filename}. Chatbot sẽ không hoạt động.")


In [ ]:
import requests
import unicodedata
from bs4 import BeautifulSoup
import re
import os

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
def remove_symbols_and_emojis(text):
    """Loại bỏ các icon và emoji phổ biến."""
    text = text.replace("🔰", "")
    text = text.replace("🔶", "")
    text = text.replace("🔸", "")
    
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        "\U0001FA00-\U0001FA6F"  # Chess Symbols & More
        "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "\U00002700-\U000027BF"  # Dingbats
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

def clean_text_content(text):
    # Lược bỏ đoạn từ "🔰 Phương thức tuyển sinh:" đến trước "🔰 Tổ hợp xét tuyển:"
    text = re.sub(
        r"Phương thức tuyển sinh:.*?(?=Tổ hợp xét tuyển:)", 
        "", 
        text, 
        flags=re.DOTALL
    )
    text = re.sub(r'\n{2,}', '\n', text)
    # Loại bỏ footer (nếu có)
    # text = re.split(r"---", text, flags=re.IGNORECASE)[0]
    text = re.split(r"5. QUYỀN LỢI CỦA NGƯỜI HỌC", text, flags=re.IGNORECASE)[0]
    # Loại bỏ icon và emoji
    text = remove_symbols_and_emojis(text)
    # Thay thế tab bằng dấu cách
    text = text.replace('\t', ' ').strip()
    return text.strip()

def remove_accents(text):
    """Tạo tên file .txt loại bỏ dấu tiếng Việt."""
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

def slugify(text):
    """Chuyển text thành tên file hợp lệ (slug)."""
    text = remove_accents(text)
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s-]', '', text) # Chỉ giữ chữ, số, khoảng trắng, gạch ngang
    text = re.sub(r'\s+', ' ', text) # Thay khoảng trắng bằng _
    text = re.sub(r'\n\n+', '\n', text) # Thay nhiều \n\n bằng 1 \n
    return text.strip()

def crawl_page(url):
    print(f">>> Đang crawl: {url}")
    try:
        response = requests.get(url, headers = headers, timeout=15)
        response.raise_for_status() # Báo lỗi nếu request không thành công
        response.encoding = "utf-8"
        soup = BeautifulSoup(response.text, "html.parser")

        title_tag = soup.find("h1", class_="article-title")
        title = title_tag.get_text(strip=True) if title_tag else "unknown_title"

        main_table = soup.find("table", class_="MsoNormalTable")
        
        raw_text = ""
        if main_table:
            print("    -> Tìm thấy MsoNormalTable. Đang lấy text...")
            raw_text = main_table.get_text(strip=False)
        else:
            print("    -> Không thấy MsoNormalTable, thử 'post-body'...")
            post_body = soup.find("div", class_="post-body")
            if post_body:
                raw_text = post_body.get_text(strip=False)
            else:
                print(f"    [!] Không tìm thấy nội dung chính tại: {url}")
                return None # Trả về None nếu không lấy được nội dung

        noidung = clean_text_content(raw_text)

        # Tìm Mã ngành bằng Regex
        ma_nganh = None
        match = re.search(r"MÃ NGÀNH:\s*(\d{7})", noidung, re.IGNORECASE)
        if match:
            ma_nganh = match.group(1)
            print(f"    -> Tìm thấy Mã ngành: {ma_nganh}")
        else:
            print("    -> Không tìm thấy Mã ngành bằng Regex.")
            
        return {            
            # "url": url,
            "ma_nganh": ma_nganh,
            "title": title,
            "noidung": noidung
        }
    except requests.exceptions.RequestException as e:
        print(f"[!] Lỗi Request khi crawl {url}: {e}")
        return None
    except Exception as e:
        print(f"[!] Lỗi không xác định khi crawl {url}: {e}")
        return None

def crawl_all(urls, output_dir="data"):
    """Crawl tất cả URL và lưu vào thư mục 'data'."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for idx, url in enumerate(urls):
        data = crawl_page(url)
        if data and data["noidung"]: # Chỉ lưu nếu có dữ liệu
            filename = f"{slugify(data['title'])}.txt"
            path = os.path.join(output_dir, filename)

            with open(path, "w", encoding="utf-8") as f:
                # f.write(f"URL: {data['url']}\n")
                # f.write(f"Mã ngành: {data['ma_nganh'] if data['ma_nganh'] else 'Không tìm thấy'}\n\n")
                f.write(data["noidung"])

            print(f"     Đã lưu: {filename}")
        else:
            print(f"     Bỏ qua URL (Không có dữ liệu): {url}")
        print("-" * 20) # Thêm dòng phân cách cho dễ nhìn

if __name__ == "__main__":
    danh_sach_url = [
        "https://ts.huit.edu.vn/nganh-dh/nganh-cong-nghe-thong-tin",
        "https://ts.huit.edu.vn/nganh-dh/nganh-marketing",
        "https://ts.huit.edu.vn/nganh-dh/nganh-ke-toan",
        "https://ts.huit.edu.vn/nganh-dh/nganh-cong-nghe-ky-thuat-co-dien-tu",
        "https://ts.huit.edu.vn/nganh-dh/nganh-tai-chinh-ngan-hang",
        "https://ts.huit.edu.vn/nganh-dh/nganh-cong-nghe-sinh-hoc",
        "https://ts.huit.edu.vn/nganh-dh/nganh-luat-kinh-te",
        "https://ts.huit.edu.vn/nganh-dh/nganh-logistic-va-quan-ly-chuoi-cung-ung",
        "https://ts.huit.edu.vn/nganh-dh/nganh-thuong-mai-dien-tu",
        "https://ts.huit.edu.vn/nganh-dh/nganh-cong-nghe-vat-lieu",
        "https://ts.huit.edu.vn/nganh-dh/nganh-kinh-doanh-quoc-te",
        "https://ts.huit.edu.vn/nganh-dh/nganh-cong-nghe-ky-thuat-hoa-hoc",
        "https://ts.huit.edu.vn/nganh-dh/nganh-quan-ly-tai-nguyen-va-moi-truong",
        "https://ts.huit.edu.vn/nganh-dh/nganh-quan-tri-khach-san",
        "https://ts.huit.edu.vn/nganh-dh/nganh-cong-nghe-det-may",
        "https://ts.huit.edu.vn/nganh-dh/nganh-quan-tri-kinh-doanh",
        "https://ts.huit.edu.vn/nganh-dh/nganh-quan-tri-dich-vu-du-lich-va-lu-hanh",
        "https://ts.huit.edu.vn/nganh-dh/nganh-cong-nghe-ky-thuat-dieu-khien-va-tu-dong-hoa",
        "https://ts.huit.edu.vn/nganh-dh/nganh-tai-chinh-ngan-hang",
        "https://ts.huit.edu.vn/nganh-dh/nganh-ngon-ngu-anh",
        "https://ts.huit.edu.vn/nganh-dh/nganh-an-toan-thong-tin",
    ]
    crawl_all(danh_sach_url)

In [ ]:
# Cell 5: Tạo Vector Database (Qdrant)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# Cấu hình chunk (giống như prepare_vector_db.py)
CHUNK_CONFIG_KAGGLE = {
    "default": {"size": 2000, "overlap": 500},
    "gioi-thieu-chung.txt": {"size": 600, "overlap": 90},
    "thu-tuc-nhap-hoc.txt": {"size": 500, "overlap": 80},
    "diem-chuan-2024.txt": {"size": 500, "overlap": 80}
}
def get_chunk_config_kaggle(filename):
    base_filename = os.path.basename(filename)
    return CHUNK_CONFIG_KAGGLE.get(base_filename, CHUNK_CONFIG_KAGGLE["default"])

def create_db_from_files_kaggle(data_directory, embedding_model_instance, qdrant_db_path, qdrant_collection):
    if not os.path.exists(data_directory):
        print(f"Thư mục dữ liệu {data_directory} không tồn tại")
        return None

    print("Bắt đầu tạo vector store Qdrant từ các file trong Kaggle...")
    all_chunks = []
    file_paths = [os.path.join(data_directory, f) for f in os.listdir(data_directory) if f.endswith(".txt") and os.path.isfile(os.path.join(data_directory, f))]

    if not file_paths:
        print(f"Không tìm thấy file .txt nào trong {data_directory}")
        return None

    for file_path in file_paths:
        filename = os.path.basename(file_path)
        print(f" Đang xử lý {filename}...")
        try:
            loader = TextLoader(file_path, encoding="utf-8")
            documents = loader.load()
            if not documents:
                print(f"    Không có nội dung trong file: {filename}")
                continue

            config = get_chunk_config_kaggle(filename)
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=config["size"], chunk_overlap=config["overlap"]
            )
            chunks_from_file = text_splitter.split_documents(documents)
            all_chunks.extend(chunks_from_file)
            print(f" Đã chia {filename} thành {len(chunks_from_file)} chunks.")
        except Exception as e:
            print(f" Lỗi khi xử lý file {filename}: {e}")

    if not all_chunks:
        print("Không có chunks nào được tạo. Dừng lại.")
        return None
    print(f"\nTổng số đoạn đã chia: {len(all_chunks)}")

    print("Đang tạo vector store Qdrant...")
    try:
        # Đảm bảo thư mục qdrant_kaggle_path tồn tại để Qdrant lưu trữ file
        os.makedirs(qdrant_db_path, exist_ok=True)
        print(f"Thư mục lưu trữ Qdrant: {qdrant_db_path}")

        qdrant_db = Qdrant.from_documents(
            documents=all_chunks,
            embedding=embedding_model_instance, # Sử dụng instance embedding_model đã tạo
            path=qdrant_db_path, # Cho lưu trữ cố định trong thư mục có thể ghi của Kaggle
            collection_name=qdrant_collection,
            force_recreate=True # Tạo lại nếu đã tồn tại, để đảm bảo sạch sẽ mỗi lần chạy trên Kaggle
        )
        print(f"Vector store Qdrant được tạo và lưu tại: {qdrant_db_path}, collection: {qdrant_collection}")
        return qdrant_db
    except Exception as e:
        print(f"Lỗi khi tạo Qdrant DB trên Kaggle: {e}")
        import traceback
        traceback.print_exc()
        return None

# Tạo DB Qdrant trên Kaggle
# embedding_model đã được khởi tạo ở Cell 2
db_instance_kaggle = None
if os.listdir(local_data_dir_kaggle): # Chỉ tạo DB nếu có file txt
    db_instance_kaggle = create_db_from_files_kaggle(
        local_data_dir_kaggle,
        embedding_model,
        qdrant_kaggle_path,
        qdrant_collection_name_kaggle
    )
    if db_instance_kaggle:
        print("Tạo DB Qdrant trên Kaggle thành công.")
    else:
        print("Tạo DB Qdrant trên Kaggle thất bại. Kiểm tra lỗi.")
else:
    print(f"Thư mục {local_data_dir_kaggle} trống, không tạo vector database.")

In [ ]:
# Cell 6: Logic Chatbot (Sử dụng Qdrant và thiết lập n_gpu_layers cho T4)
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Các biến đã định nghĩa:
# model_GGUF_path_kaggle (từ Cell 3)
# embedding_model (từ Cell 2)
# qdrant_kaggle_path, qdrant_collection_name_kaggle (từ Cell 2)

general_temperature_kaggle = 0.01
max_token_output_kaggle = 512
retrieval_k_value_kaggle = 4 # Số lượng tài liệu truy xuất (tương tự faiss_search)

def read_vectors_db_kaggle(persistent_path, collection_name_str, embeddings_instance):
    print(f"Đang tải vector database Qdrant từ: {persistent_path}, collection: {collection_name_str}")
    try:
        if not os.path.exists(persistent_path) or not os.listdir(persistent_path):
             print(f"LỖI: Đường dẫn Qdrant {persistent_path} không tồn tại hoặc trống. Collection có thể chưa được tạo.")
             return None

        client = QdrantClient(path=persistent_path) # Cho lưu trữ trên đĩa
        
        # Kiểm tra collection
        try:
            collection_info = client.get_collection(collection_name=collection_name_str)
            print(f"Thông tin collection Qdrant: {collection_info}")
            if collection_info.points_count == 0:
                 print(f"CẢNH BÁO: Collection Qdrant '{collection_name_str}' trống.")
        except Exception as e:
            print(f"LỖI: Không thể lấy thông tin collection '{collection_name_str}'. Lỗi: {e}")
            client.close()
            return None


        qdrant_store = Qdrant(
            client=client, # client sẽ được quản lý (đóng) bởi instance Qdrant khi không còn dùng
            collection_name=collection_name_str,
            embeddings=embeddings_instance
        )
        print("Đã kết nối tới vector database Qdrant trên Kaggle.")
        return qdrant_store
    except Exception as e:
        print(f"Lỗi khi tải Qdrant DB trên Kaggle: {e}")
        import traceback
        traceback.print_exc()
        return None

def load_llm_kaggle(model_path_gguf_kaggle):
    # Đối với Kaggle T4 GPU (thường có 16GB VRAM):
    # VinaLLaMA-7B (dựa trên Llama 2) có 32 layers.
    # - Đặt `n_gpu_layers = -1` sẽ cố gắng offload tất cả các layer có thể lên GPU.
    # - Hoặc đặt bằng số layer thực tế (ví dụ: 32 cho Llama 7B).
    # Với 16GB VRAM, một model Q5_0 7B (khoảng 4.375GB cho trọng số) nên vừa hoàn toàn trên GPU,
    # cộng thêm bộ nhớ cho context và activations.
    #
    # Nếu Kaggle cung cấp T4 x2, Llama.cpp (phiên bản Python bindings cơ bản) thường chỉ sử dụng 1 GPU
    # cho một tiến trình LLM. Nếu bạn muốn sử dụng cả 2 GPU, cần các thiết lập phức tạp hơn
    # (ví dụ: model parallelism với thư viện khác hoặc chạy 2 instance LlamaCpp riêng biệt).
    # Giả sử ở đây chúng ta dùng 1 GPU T4.
    n_gpu_layers_t4 = -1 # Offload tất cả các layer có thể lên GPU. Hoặc thử 32.

    if not model_path_gguf_kaggle or not os.path.exists(model_path_gguf_kaggle):
        print(f"Lỗi: Không tìm thấy file model GGUF tại {model_path_gguf_kaggle}")
        return None

    llm = LlamaCpp(
        model_path=model_path_gguf_kaggle,
        n_gpu_layers=n_gpu_layers_t4,
        n_ctx=4096,
        n_batch=512, # Có thể cần giảm nếu gặp vấn đề về VRAM, nhưng 512 thường ổn cho T4
        max_tokens=max_token_output_kaggle,
        temperature=general_temperature_kaggle,
        top_p=0.9,
        use_mlock=True,
        use_mmap=True,
        f16_kv=True, # T4 hỗ trợ tốt FP16
        verbose=True # Đặt True để xem thêm thông tin, False để output gọn hơn
    )
    return llm

def create_prompt_kaggle(template_str):
    return PromptTemplate(template=template_str, input_variables=["context", "question"])

def create_qa_chain_kaggle(prompt_obj, llm_obj, db_obj):
    retriever = db_obj.as_retriever(search_kwargs={"k": retrieval_k_value_kaggle})
    chain = RetrievalQA.from_chain_type(
        llm=llm_obj,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True, # Hữu ích để debug, xem context nào được lấy
        chain_type_kwargs={'prompt': prompt_obj}
    )
    return chain

# --- Thực thi chính cho truy vấn chatbot ---
if model_GGUF_path_kaggle and os.path.exists(model_GGUF_path_kaggle) and db_instance_kaggle:
    print("Bắt đầu đọc database Qdrant (Kaggle) cho truy vấn...")
    # Sử dụng cùng instance embedding_model đã dùng để tạo DB
    # Hoặc có thể gọi lại read_vectors_db_kaggle nếu db_instance_kaggle không được trả về đúng cách
    # db_for_query_kaggle = read_vectors_db_kaggle(qdrant_kaggle_path, qdrant_collection_name_kaggle, embedding_model)
    db_for_query_kaggle = db_instance_kaggle # Nếu create_db_from_files_kaggle trả về instance Qdrant

    if db_for_query_kaggle:
        print("Đã tải/sử dụng thành công vector database Qdrant (Kaggle).")

        question_kaggle = "Học phí 1 tín chỉ là bao nhiêu?" # Câu hỏi thử nghiệm của bạn
        print(f"\nCâu hỏi: {question_kaggle}")

        print("\nKiểm tra truy xuất context từ câu hỏi (Qdrant)...")
        retrieved_docs_kaggle = db_for_query_kaggle.similarity_search(question_kaggle, k=retrieval_k_value_kaggle)
        for i, doc_item in enumerate(retrieved_docs_kaggle):
            print(f"\n--- Kết quả Qdrant (debug) {i+1} ---\n{doc_item.page_content}\nMetadata: {doc_item.metadata}\n")


        print("Tải mô hình LLM (Kaggle)...")
        llm_instance_kaggle = load_llm_kaggle(model_GGUF_path_kaggle)

        if llm_instance_kaggle:
            template_rag_kaggle = """<|im_start|>system
                Bạn là một trợ lý tư vấn tuyển sinh. Sử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy những câu đại loại như "Từ tận đáy lòng, tôi thực sự xin lỗi, tôi không tìm thấy thông tin liên quan trong dữ liệu", đừng cố bịa ra câu trả lời, cũng đừng hallucinate. Trả lời câu hỏi bên dưới:
                {context}<|im_end|>
                <|im_start|>user
                {question}<|im_end|>
                <|im_start|>assistant"""

            print("\nTạo prompt từ template...")
            prompt_instance_kaggle = create_prompt_kaggle(template_rag_kaggle)

            print("\nTạo chuỗi LLM với truy xuất RAG (Qdrant)...")
            llm_chain_instance_kaggle = create_qa_chain_kaggle(prompt_instance_kaggle, llm_instance_kaggle, db_for_query_kaggle)

            print("\nBắt đầu trả lời câu hỏi...")
            answer_dict_kaggle = llm_chain_instance_kaggle.invoke({"query": question_kaggle})

            print("\n--- Câu trả lời từ LLM ---")
            print(answer_dict_kaggle.get("result", "Không có kết quả"))

            print("\n--- Nguồn tài liệu tham khảo (nếu có) ---")
            if "source_documents" in answer_dict_kaggle and answer_dict_kaggle["source_documents"]:
                for idx, s_doc in enumerate(answer_dict_kaggle["source_documents"]):
                    print(f"Nguồn {idx+1}: Metadata: {s_doc.metadata}")
                    print(f"{s_doc.page_content[:300]}...\n") # In một đoạn trích
            else:
                print("Không có nguồn tài liệu nào được trả về hoặc chúng trống.")
            print("-" * 30)
        else:
            print("[!] Không thể tải mô hình LLM trên Kaggle.")
    else:
        print("[!] Không thể tải/sử dụng vector database Qdrant trên Kaggle.")
elif not model_GGUF_path_kaggle or not os.path.exists(model_GGUF_path_kaggle):
    print(f"[!] Không thể thực hiện RAG do model GGUF không được tìm thấy hoặc không tải được: {model_GGUF_path_kaggle}")
elif not db_instance_kaggle:
    print(f"[!] Không thể thực hiện RAG do vector database (Qdrant) chưa được tạo hoặc tải thành công.")


In [ ]:
question = "Học phí 1 tín chỉ là bao nhiêu"
faiss_search = 4  # Số lượng kết quả trả về từ FAISS
print("Bắt đầu đọc database...")
db = read_vectors_db()
print("Đã tải thành công vector database.")

print("Kiểm tra truy xuất context từ câu hỏi")
docs = db.similarity_search(question, k=faiss_search)
for i, doc in enumerate(docs):
    print(f"\n--- Kết quả FAISS {i+1} ---\n{doc.page_content}\n")

print("Tải mô hình LLM...")
llm = load_llm(model_GGUF)

template = """<|im_start|>system\n
Bạn là một trợ lý tư vấn tuyển sinh. Sử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy những câu đại loại như "Từ tận đáy lòng, tôi thực sự xin lỗi, tôi không tìm thấy thông tin liên quan trong dữ liệu", đừng cố bịa ra câu trả lời, cũng đừng hallucinate. Trả lời câu hỏi bên dưới\n
{context}<|im_end|>\n
<|im_start|>user\n
{question}<|im_end|>\n
<|im_start|>assistant"""

print("Tạo prompt từ template...")
prompt = create_prompt(template)

print("Tạo chuỗi LLM với truy xuất RAG...")
llm_chain = create_qa_chain(prompt, llm, db)

# answer = llm_chain.invoke({"query": question})
answer = llm_chain.invoke({"query": question})
print(answer)